In [114]:
import pandas as pd
import json
from pandas import json_normalize
import numpy as np
from pandasql import sqldf


In [101]:
#Loading the json files
r = pd.read_json('C:\\Users\\Karthoz\\OneDrive\\Documents\\FETCH\\receipts\\receipts.json', lines=True)
u = pd.read_json('C:\\Users\\Karthoz\\OneDrive\\Documents\\FETCH\\users\\users.json', lines=True)
b = pd.read_json('C:\\Users\\Karthoz\\OneDrive\\Documents\\FETCH\\brands\\brands.json', lines=True)


In [102]:
#Creating copies to make changes/clean in the data
df1=r.copy()
df2=u.copy()
df3=b.copy()

In [112]:
print("receipts: ",df1.shape)
print("users: ",df2.shape)
print("brands: ",df3.shape)


receipts:  (1119, 15)
users:  (495, 7)
brands:  (1167, 8)


Cleaning the data to have in suitable format and removing nesting

In [119]:
#cleaning the data/ unnesting the columns
df1['id']=pd.json_normalize(df1['_id'])
df1['create_Date']=pd.json_normalize(df1['createDate'])
df1['date_Scanned']=pd.json_normalize(df1['dateScanned'])
df1['finished_Date']=pd.json_normalize(df1['finishedDate'])
df1['modify_Date']=pd.json_normalize(df1['modifyDate'])
df1['pointsAwarded_Date']=pd.json_normalize(df1['pointsAwardedDate'])
df1['purchase_Date']=pd.json_normalize(df1['purchaseDate'])

df1=df1.drop(['createDate','_id','dateScanned','finishedDate','pointsAwardedDate','purchaseDate','modifyDate'],axis=1)

df1.head()

,bonusPointsEarned,bonusPointsEarnedReason,pointsEarned,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId,id,create_Date,date_Scanned,finished_Date,modify_Date,pointsAwarded_Date,purchase_Date
0,500.0,"Receipt number 2 completed, bonus point schedu...",500.0,5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.0,5ff1e1eacfcf6c399c274ae6,5ff1e1eb0a720f0523000575,1609687531000,1609687531000,1.609688e+12,1609687536000,1.609688e+12,1.609632e+12
1,150.0,"Receipt number 5 completed, bonus point schedu...",150.0,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.0,5ff1e194b6a9d73a3a9f1052,5ff1e1bb0a720f052300056b,1609687483000,1609687483000,1.609687e+12,1609687488000,1.609687e+12,1.609601e+12
2,5.0,All-receipts receipt bonus,5.0,1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.0,5ff1e1f1cfcf6c399c274b0b,5ff1e1f10a720f052300057a,1609687537000,1609687537000,NaN,1609687542000,NaN,1.609632e+12
3,5.0,All-receipts receipt bonus,5.0,4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.0,5ff1e1eacfcf6c399c274ae6,5ff1e1ee0a7214ada100056f,1609687534000,1609687534000,1.609688e+12,1609687539000,1.609688e+12,1.609632e+12
4,5.0,All-receipts receipt bonus,5.0,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.0,5ff1e194b6a9d73a3a9f1052,5ff1e1d20a7214ada1000561,1609687506000,1609687506000,1.609688e+12,1609687511000,1.609688e+12,1.609601e+12


In [121]:
rows = []

# Iterate through each row in the DataFrame
for index, row in df1.iterrows():
    # Get the rewardsReceiptItemList for the current row
    items_list = row['rewardsReceiptItemList']
   
    # If items_list is a list, create a row for each item
    if isinstance(items_list, list):
        for item in items_list:
            # Create a copy of the current row
            new_row = row.copy()
            # Update the rewardsReceiptItemList with the current item
            new_row['rewardsReceiptItemList'] = item
            # Append the new row to the list of rows
            rows.append(new_row)

# Create a new DataFrame from the list of rows
result_df = pd.DataFrame(rows)

# Display the size of resulting DataFrame, each row has each item
print(result_df.shape)


(6941, 15)


In [167]:
df_new = pd.json_normalize(result_df['rewardsReceiptItemList'])
df_new = df_new.rename(columns={'pointsEarned': 'pointsEarned_item'})


In [128]:
#Final receipts table
df_new.reset_index(inplace=True, drop=True)
result_df.reset_index(inplace=True, drop=True)
receipts = pd.concat([result_df,df_new],axis=1)
receipts= receipts.drop(['rewardsReceiptItemList'],axis=1)
# receipts.columns.values[2] = 'pointsEarned_receipt'


(6941, 48)

In [135]:
df2['id']=pd.json_normalize(df2['_id'])
df2['created_Date']=pd.json_normalize(df2['createdDate'])
df2['last_Login']=pd.json_normalize(df2['lastLogin'])
df2=df2.drop(['createdDate','_id','lastLogin'],axis=1)

df2.head()
users=df2.copy()


,active,role,signUpSource,state,id,created_Date,last_Login
0,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,1609687444800,1.609688e+12
1,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,1609687444800,1.609688e+12
2,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,1609687444800,1.609688e+12
3,True,consumer,Email,WI,5ff1e1eacfcf6c399c274ae6,1609687530554,1.609688e+12
4,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,1609687444800,1.609688e+12


In [149]:
df3.shape

(1167, 9)

In [155]:
df3['id']=pd.json_normalize(df3['_id'])
df_cpg=df3.cpg.apply(pd.Series)
df_cpg['$ref'].unique()  #we can delete this column as it has only 2 values and probably both are same and its typo 
df3['cpg_id']=pd.json_normalize(df_cpg['$id'])
df3['cpg_ref']=df_cpg['$ref']

df3.drop(['_id','cpg'],axis=1)
brands=df3.copy()

Creating Fact table : Rewards and Dimension Tables : Receipts, Users, Brands

In [184]:
receipts['barcode'] = receipts['barcode'].fillna(0)
brands['barcode']=brands['barcode'].astype(str)


In [190]:
merged_df = pd.merge(receipts, users, left_on='userId', right_on='id', how='inner')
merged_df = pd.merge(merged_df, brands, left_on='barcode', right_on='barcode', how='inner')

rewards = merged_df[['id_x', 'userId', 'id_y', 'totalSpent', 'itemPrice', 'barcode', 'finalPrice', 'purchasedItemCount']]
rewards.columns = ['Receipt_id', 'User_id', 'Brand_id', 'Totalspent', 'Itemprice', 'Barcode', 'Finalprice', 'purchasedItemCount']

print(rewards.head())

                 Receipt_id                   User_id  \
0  600206000a720f05f3000087  6000b75bbe5fc96dfee1d4d3   
1  600206000a720f05f3000087  6000b75bbe5fc96dfee1d4d3   
2  600206000a720f05f3000087  6000b75bbe5fc96dfee1d4d3   
3  600206000a720f05f3000087  6000b75bbe5fc96dfee1d4d3   
4  600206000a720f05f3000087  6000b75bbe5fc96dfee1d4d3   

                   Brand_id  Totalspent Itemprice       Barcode Finalprice  \
0  6000b75bbe5fc96dfee1d4d3      271.63      5.58  511111204206       5.58   
1  6000b75bbe5fc96dfee1d4d3      271.63      5.58  511111204206       5.58   
2  6000b75bbe5fc96dfee1d4d3      271.63      5.58  511111204206       5.58   
3  6000b75bbe5fc96dfee1d4d3      271.63      5.58  511111204206       5.58   
4  6000b75bbe5fc96dfee1d4d3      271.63      5.58  511111204206       5.58   

   purchasedItemCount  
0                86.0  
1                86.0  
2                86.0  
3                86.0  
4                86.0  


In [191]:
receipts = receipts.drop(['totalSpent', 'itemPrice', 'finalPrice', 'purchasedItemCount'],axis=1)



In [192]:
#Fact and dimension tables
print("receipts: ",receipts.shape)
print("users: ",users.shape)
print("brands: ",brands.shape)
print("rewards: ",rewards.shape)


receipts:  (6941, 44)
users:  (495, 7)
brands:  (1167, 11)
rewards:  (92, 8)


Data Quality 

In [194]:
receipts.info()
brands.info()
users.info()
rewards.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6941 entries, 0 to 6940
Data columns (total 44 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   bonusPointsEarned                   5978 non-null   float64
 1   bonusPointsEarnedReason             5978 non-null   object 
 2   pointsEarned_receipt                6251 non-null   float64
 3   rewardsReceiptStatus                6941 non-null   object 
 4   userId                              6941 non-null   object 
 5   id                                  6941 non-null   object 
 6   create_Date                         6941 non-null   int64  
 7   date_Scanned                        6941 non-null   int64  
 8   finished_Date                       5967 non-null   float64
 9   modify_Date                         6941 non-null   int64  
 10  pointsAwarded_Date                  6078 non-null   float64
 11  purchase_Date                       6920 no

In [197]:
print(receipts.describe(include='object'))
print(users.describe(include='object'))
print(brands.describe(include='object'))
print(rewards.describe(include='object'))

                                  bonusPointsEarnedReason  \
count                                                5978   
unique                                                  9   
top     Receipt number 1 completed, bonus point schedu...   
freq                                                 4605   

       rewardsReceiptStatus                    userId  \
count                  6941                      6941   
unique                    4                       246   
top                FINISHED  600f29a64329897eac239049   
freq                   5918                       459   

                              id  barcode     description needsFetchReview  \
count                       6941     6941            6560              813   
unique                       679      569            1889                2   
top     600f2fc80a720f0535000030        0  ITEM NOT FOUND            False   
freq                         459     3851             173              594   

       partnerIte

SQL Queries ( 2 examples)

In [ ]:
# Query definition
query = """ select r.rewardsReceiptStatus
     , Sum(rw.purchasedItemCount) as totalcount
  from Receipts r 
  JOIN Rewards rw on rw.receiptid=r.id
group
    by rewardsReceiptStatus
having Sum(rw.purchasedItemCount) =
     ( select max(totalcount) as highest_count
         from (
	 SELECT
    r.rewardsReceiptStatus,
    Sum(rw.purchasedItemCount) AS totalcount
FROM
    Receipts 
	JOIN Rewards rw on rw.receiptid=r.id
WHERE
    r.rewardsReceiptStatus IN ('Accepted', 'Rejected')
GROUP BY
    r.rewardsReceiptStatus
	) as t );
        """


In [ ]:
query = """ SELECT
    b.id AS brand_id,
    b.name AS brand_name,
    COUNT(r.id) AS transaction_count
FROM
    brand b
JOIN
    rewards rw ON b.barcode = rw.barcode
JOIN
    receipts r ON rw.receiptid = r.id
JOIN
    RecentUsers ru ON rw.userid = ru.user_id
GROUP BY
    brand_id, brand_name
ORDER BY
    transaction_count DESC
LIMIT 1;
        """

In [ ]:
# Query execution
output = sqldf(query)
output